<h2 style="color: #0000FF;">CALCULADORA DE PRECIOS</h2>

In [1]:
%matplotlib inline

# 📚 Basic Libraries
import numpy as np # operaciones matemáticas (numerical python)
import pandas as pd # manipulación de datos
import warnings # nobody likes warnings

# 📊 Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# 🤖 Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.linear_model import Ridge

import http.client
import json
from bs4 import BeautifulSoup
import pandas as pd

import plotly.express as px

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
data = pd.read_csv('datos_coches_limpios.csv')

In [3]:
numeric_data = data.select_dtypes(include='number')

In [4]:
numeric_data.columns

Index(['year', 'condition', 'odometer', 'market_price', 'car age', 'price'], dtype='object')

<h2 style="color: #0000FF;">¿QUÉ ES UNA CALCULADORA DE PRECIOS?</h2>

Como hemos podido ver el modelo que mejor predice los precios es el modelo RIDGE y aun que no hay una mejoría notable en la precisión de esta predicción cuando escalamos los datos, la desviación si que disminuye.

Escalamos los datos con MinMaxScaler

Ademas mejoramos también el modelo a través de los hiperparametros 

Por último añadimos intereacciones polinómicas entre las características existentes para encontrar relaciones no lineales.



In [5]:
# Separar características (X) y etiquetas (y)
X = numeric_data.drop('price', axis=1)  
y = numeric_data['price']


In [6]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Normalizar características
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
# Entrenar el modelo Ridge y seleccionar los mejores hiperparámetros
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(Ridge(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100]})

In [9]:
# Obtener los mejores hiperparámetros
best_alpha = grid_search.best_params_['alpha']

In [10]:
# Funciones polinómicas
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)



In [11]:
# Entrenar el modelo Ridge con los mejores hiperparámetros y características polinómicas
ridge_best = Ridge(alpha=best_alpha)
ridge_best.fit(X_train_poly, y_train)


Ridge(alpha=0.001)

In [12]:
# Realizar predicciones en el conjunto de prueba para evaluar el rendimiento
predictions_ridge = ridge_best.predict(X_test_poly)
r2_3_2 = r2_score(y_test, predictions_ridge)
RMSE_3_2 = mean_squared_error(y_test, predictions_ridge, squared=False)
MSE_3_2 = mean_squared_error(y_test, predictions_ridge)
MAE_3_2 = mean_absolute_error(y_test, predictions_ridge)

In [13]:
print("R2 =", round(r2_3_2, 4))
print("RMSE =", round(RMSE_3_2, 4))
print("MSE =", round(MSE_3_2, 4))
print("MAE =", round(MAE_3_2, 4))


R2 = 0.9719
RMSE = 1596.9962
MSE = 2550396.8786
MAE = 1015.9859


In [14]:
# Función para preprocesar la entrada del usuario
def preprocess_user_input(year, condition, odometer, car_age, market_price):
    user_input_df = pd.DataFrame({
        'year': [year],
        'condition': [condition],
        'odometer': [odometer],
        'market_price': [market_price],
        'car age': [car_age]
    })

    # Reorganizar las columnas en el orden correcto
    user_input_df = user_input_df[['year', 'condition', 'odometer', 'market_price', 'car age']]

    # Normalizar características
    user_input_scaled = scaler.transform(user_input_df)

    # Aplicar funciones polinómicas
    user_input_poly = poly.transform(user_input_scaled)

    return user_input_poly



In [15]:
import webbrowser

while True:
    try:
        year = int(input("Ingrese el año del vehículo: "))
        
        # Explicación de la condición del vehículo
        print("Por favor, indique la condición del vehículo en una escala del 1 al 100:")
        print(" - 100: Sin apenas uso y en perfecto estado.")
        print(" - 80: Con algún rasguño por fuera o por dentro.")
        print(" - 60: Con algún golpe.")
        print(" - 40: Alguna luz no funciona o tiene más de un golpe.")
        print(" - 20: Alguna luz no funciona además tiene más de un golpe y por dentro no esta en buen estado.")
        print(" - 0: En condición muy deficiente.")
        
        condition = float(input("Ingrese la condición del vehículo: "))        
        odometer = float(input("Ingrese el odómetro del vehículo (en km): "))
        car_age = float(input("Ingrese los años de uso del vehículo: "))
        
        # Preguntar si el usuario conoce el precio de mercado
        know_market_price = input("¿Conoce el precio de mercado del vehículo? (si/no): ").lower()

        if know_market_price == 'si':
            market_price = float(input("Ingrese el precio de mercado del vehículo (en dólares): "))
        else:
            # Abrir una página web de coches de segunda mano si no conoce el precio de mercado
            webbrowser.open("https://www.ooyyo.com/estados+unidos/coches-usados-anuncios/c=CDA31D7114D38568111BF66FAA651453/")  
            
            # Preguntar nuevamente por el precio de mercado
            know_market_price = input("¿Conoce el precio de mercado del vehículo ahora? (si/no): ").lower()

            if know_market_price == 'si':
                market_price = float(input("Ingrese el precio de mercado del vehículo (en dólares): "))
            else:
                break  # Salir del bucle si no desean continuar

        break  # Salir del bucle si todas las entradas son válidas

    except ValueError:
        print("Error: Ingrese valores numéricos válidos.")



Ingrese el año del vehículo:  2020


Por favor, indique la condición del vehículo en una escala del 1 al 100:
 - 100: Sin apenas uso y en perfecto estado.
 - 80: Con algún rasguño por fuera o por dentro.
 - 60: Con algún golpe.
 - 40: Alguna luz no funciona o tiene más de un golpe.
 - 20: Alguna luz no funciona además tiene más de un golpe y por dentro no esta en buen estado.
 - 0: En condición muy deficiente.


Ingrese la condición del vehículo:  40
Ingrese el odómetro del vehículo (en km):  65000
Ingrese los años de uso del vehículo:  2
¿Conoce el precio de mercado del vehículo? (si/no):  no
¿Conoce el precio de mercado del vehículo ahora? (si/no):  si
Ingrese el precio de mercado del vehículo (en dólares):  20000


In [16]:
condition= condition*0.8
market_price=market_price*0.8

In [17]:
market_price

16000.0

In [18]:
# Preprocesar datos de entrada del usuario
user_input_poly = preprocess_user_input(year, condition, odometer, car_age, market_price)

In [19]:
# Realizar predicción con el modelo entrenado
predicted_price = ridge_best.predict(user_input_poly)

In [20]:
# Imprimir resultados
print(f"El precio estimado del vehículo es: {predicted_price[0]:.2f}")

El precio estimado del vehículo es: 16045.41


## 